#### resize images so they are a uniform size

In [3]:
# packages
from PIL import Image as img
import os
import pandas as pd
import shutil

# set wd
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/PyTorch/Fracture/')

In [4]:
# empty folders to avoid duplicates as there has been a few iterations of naming schemes used
# empty the folder

try:
    shutil.rmtree('./images/fracture_resize')
    # recreate the empty directory
    os.makedirs('./images/fracture_resize')
    print("Folder emptied successfully")
except Exception as e:
    print(f"Error occurred: {e}")

# empty the folder
try:
    shutil.rmtree('./images/non_fractured_resize')
    # recreate the empty directory
    os.makedirs('./images/non_fractured_resize')
    print("Folder emptied successfully")
except Exception as e:
    print(f"Error occurred: {e}")

Folder emptied successfully
Folder emptied successfully


In [5]:
# Resize the images to 224x224, which is the most common size of images. 
# Images that are not this size also only have to downscale which is more optimal
# Also added more details to the file name to make sure everyname was unqiue

# function to resize
def resize(imagePath, targetSize, save_folder, type):
    # open and resize
    image = img.open(imagePath)
    image = image.resize(targetSize)
    
    # get base filename and add type to ensure unique file names
    base_name = os.path.splitext(os.path.basename(imagePath))[0]  # get filename without extension
    extension = os.path.splitext(imagePath)[1]  # get extension
    new_name = f"{base_name}_{type}{extension}"
    
    # save the new image 
    save_path = os.path.join(save_folder, new_name)
    image.save(save_path) 
    

# iterate over elements in folder | no need here, all images are fine
for filename in os.listdir('./images/original_data/Fractured'):
    file_path = os.path.join('./images/original_data/Fractured', filename)
    resize(file_path, (224,224), './images/fracture_resize', "frac")

# had to implement try and except because a few images were corrupted in this folder 
for filename in os.listdir('./images/original_data/Non_fractured'):
    try:
        file_path = os.path.join('./images/original_data/Non_fractured', filename)
        resize(file_path, (224,224),'./images/non_fractured_resize', "nfrac")
    except:
        # if there's a file error, print the error and continue
        print(f"Error processing {filename}, skipping...")
        continue

Error processing IMG0004028.jpg, skipping...
Error processing IMG0004029.jpg, skipping...
Error processing IMG0004036.jpg, skipping...
Error processing IMG0004070.jpg, skipping...
Error processing IMG0004073.jpg, skipping...
Error processing IMG0004076.jpg, skipping...
Error processing IMG0004079.jpg, skipping...
Error processing IMG0004084.jpg, skipping...
Error processing IMG0004092.jpg, skipping...
Error processing IMG0004098.jpg, skipping...
Error processing IMG0004100.jpg, skipping...
Error processing IMG0004109.jpg, skipping...
Error processing IMG0004119.jpg, skipping...
Error processing IMG0004120.jpg, skipping...
Error processing IMG0004121.jpg, skipping...
Error processing IMG0004122.jpg, skipping...
Error processing IMG0004123.jpg, skipping...
Error processing IMG0004129.jpg, skipping...
Error processing IMG0004130.jpg, skipping...
Error processing IMG0004134.jpg, skipping...
Error processing IMG0004142.jpg, skipping...
Error processing IMG0004143.jpg, skipping...
Error proc

In [ ]:
from PIL import ImageOps

def convert_to_grayscale(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        image = img.open(file_path)
        
        # Check if image is RGB (3 channels)
        if image.mode == 'RGB':
            # Convert to grayscale
            gray_image = ImageOps.grayscale(image)
            # Save back to the same location
            gray_image.save(file_path)
            print(f"{image} converted to greyscale")

# Convert images in both folders
convert_to_grayscale('./images/fracture_resize')
convert_to_grayscale('./images/non_fractured_resize')

In [7]:
# create a matching csv to prep for pytorch data loading
data = {'name': [], 'class': []}

# iterate over file names in fracture images
for filename in os.listdir('./images/fracture_resize'):
    data['name'].append(filename)
    data['class'].append(1) # FRACTURE = 1

for filename in os.listdir('./images/non_fractured_resize'):
    data['name'].append(filename)
    data['class'].append(0) # NO FRACTURE = 0

# create dataframe
class_id = pd.DataFrame(data)
class_id.to_csv('./images/class_ids.csv', index=False) # note the slight class imbalance that potentially needs to be accounted for

In [8]:
# create a new directory for combined data
try:
    shutil.rmtree('./images/resize_data')
    os.makedirs('./images/resize_data')
    print("Created new resize_data directory")
except Exception as e:
    os.makedirs('./images/resize_data', exist_ok=True)
    print("Using existing resize_data directory")

# copy fracture images
for img_file in os.listdir('./images/fracture_resize'):
    src = os.path.join('./images/fracture_resize', img_file)
    dst = os.path.join('./images/resize_data', img_file)
    shutil.copy2(src, dst)

# copy non-fracture images
for img_file in os.listdir('./images/non_fractured_resize'):
    src = os.path.join('./images/non_fractured_resize', img_file)
    dst = os.path.join('./images/resize_data', img_file)
    shutil.copy2(src, dst)

print(f"Total images in resize_data: {len(os.listdir('./images/resize_data'))}")

Created new resize_data directory
Total images in resize_data: 6151
